# Parte 04

Nessa parte os modelos criados anteriormente serão utilizados para realizar predições. Para isso, eles devem ser
registrados no TFX. Para efetuar as predições, os dados utilizados no treinamento desses modelos serão inseridos
no SAVIME, o qual ficará encarregado de enviar e receber os dados para/de TFX. 

In [51]:
import os
import sys

# Necessário mudar o diretório de trabalho para o nível mais acima
if not 'notebooks' in os.listdir('.'):
    current_dir = os.path.abspath(os.getcwd())
    parent_dir = os.path.dirname(current_dir)
    os.chdir(parent_dir)

# Inserir aqui o caminho do arquivo de dados: um json contendo informações a respeito 
# da partição de x e y utilizada na parte 01.
data_fp = 'saved_models_arima/data.json'

# Configuração do host e porta em que o SAVIME está escutando
savime_host = '127.0.0.1'
savime_port = 65000

# Configuração TFX
tfx_host = 'localhost'
tfx_port = 8501

# Diretório de dados
data_dir = 'data'

# Local do array de temperaturas
dataset_path = os.path.join(data_dir, 'tiny-dataset.hdf5')

In [52]:
%load_ext autoreload
%autoreload 2
%matplotlib agg

from IPython.display import HTML


import json
import h5py
import numpy as np
import seaborn as sns
import tensorflow as tf

from src.animation import animate_heat_map
from src.predictor_consumer import PredictionConsumer

# Savime imports
import pysavime
from pysavime.util.converter import DataVariableBlockConverter

sns.set_context('notebook')
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette("Paired"))

tf.get_logger().setLevel('ERROR')

with open(data_fp, 'r') as _in:
    data = json.load(_in)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


A primeira etapa a ser realizada é converter os dados para um formato processável para o SAVIME.

In [53]:
with h5py.File(dataset_path, 'r') as in_:
    array = in_['real'][...]
    
# Especificar dimensões
time_series = ('time_series', range(array.shape[0]))
time_step = ('time_step', range(array.shape[1]))
pos_x = ('pos_x', range(array.shape[2]))
pos_y = ('pos_y', range(array.shape[3]))

# Remover última dimensão espúria
squeezed_array = np.squeeze(array, axis=-1)

# Salvar array
temperatura_data_fp = os.path.join(data_dir, 'temperatura.data')
squeezed_array.ravel().astype('float64').tofile(temperatura_data_fp)

Também é nessário fazer a divisão do conjunto de dados de entrada em x e y. Como dito na parte anterior, cada série temporal possuí 10 instantes de tempo. Além disso, os modelos foram treinados a prever o décimo instante de tempo a partir dos nove anteriores. A critério de exemplo, selecionamos abaixo um grupo de séries temporais para realizar a predição de temperatura.

In [54]:
# Seleciona-se apenas um grupo para predição. 
chosen_model_name = data['model']
chosen_group_ix = 0
x = squeezed_array[[chosen_group_ix], :-1]
y = squeezed_array[[chosen_group_ix], 1:]

pc = PredictionConsumer(host=tfx_host, port=tfx_port, model_name=chosen_model_name)
y_hat = pc.predict(x)

In [55]:
anim_y = animate_heat_map(np.squeeze(y,axis=0))
anim_y_html = anim_y.to_html5_video()
anim_yhat = animate_heat_map(np.squeeze(y_hat,axis=0))
anim_yhat_html = anim_yhat.to_html5_video()
HTML(f'<div style="float: left;"> {anim_y_html} </div><div style="float: left;"> {anim_yhat_html} </div>')

In [56]:
num_models = 25
num_groups, num_time_steps, num_pos_x, num_pos_y = squeezed_array.shape  

# Define o dataset com as temperaturas a ser registrado no SAVIME.
dataset =  pysavime.define.file_dataset('temperature_data', temperatura_data_fp, 'double')
print('- Dataset CREATE query:', dataset.create_query_str())

# Define o esquema do tar
group_dim = pysavime.define.implicit_tar_dimension('group', 'int32', 0, num_groups - 1)
time_step_dim = pysavime.define.implicit_tar_dimension('time_step', 'int32', 0, num_time_steps - 1)
pos_x_dim = pysavime.define.implicit_tar_dimension('pos_x', 'int32', 0, num_pos_x - 1)
pos_y_dim = pysavime.define.implicit_tar_dimension('pos_y', 'int32', 0, num_pos_y - 1)
temperature = pysavime.define.tar_attribute('temperature', 'double')

dims = [group_dim, time_step_dim, pos_x_dim, pos_y_dim]
attributes = [temperature]
tar = pysavime.define.tar('temperatures_tar', dims, attributes)
print('- Tar CREATE query:', tar.create_query_str())

# Define o subtar único responsável por registrar o dataset no tar criado anteriormente.
group_dim_sub = pysavime.define.ordered_subtar_dimension(group_dim, 0, num_groups - 1, True)
time_step_dim_sub = pysavime.define.ordered_subtar_dimension(time_step_dim, 0, num_time_steps - 1, True)
pos_x_dim_sub = pysavime.define.ordered_subtar_dimension(pos_x_dim, 0, num_pos_x - 1, True)
pos_y_dim_sub = pysavime.define.ordered_subtar_dimension(pos_y_dim, 0, num_pos_y - 1, True)
temperature_sub = pysavime.define.subtar_attribute(temperature, dataset)

subtar_dims = [group_dim_sub, time_step_dim_sub, pos_x_dim_sub, pos_y_dim_sub]
subtar_attrs = [temperature_sub]
subtar = pysavime.define.subtar(tar, subtar_dims, subtar_attrs)
print('- SubTar LOAD query', subtar.load_query_str())

- Dataset CREATE query: CREATE_DATASET("temperature_data:double:1", "@data/temperatura.data");
- Tar CREATE query: CREATE_TAR("temperatures_tar", "*", "implicit, group, int32, 0, 399, 1 | implicit, time_step, int32, 0, 9, 1 | implicit, pos_x, int32, 0, 34, 1 | implicit, pos_y, int32, 0, 39, 1", "temperature, double: 1");
- SubTar LOAD query LOAD_SUBTAR("temperatures_tar", "ordered, group, 0,399 | ordered, time_step, 0,9 | ordered, pos_x, 0,34 | ordered, pos_y, 0,39", "temperature, temperature_data")


In [57]:
with pysavime.Client(host='127.0.0.1', port=65000, raise_silent_error=True) as client:
    client.execute(pysavime.operator.create(dataset))
    client.execute(pysavime.operator.create(tar))
    client.execute(pysavime.operator.load(subtar))

2020-02-15 20:27:02 [Client]:ERROR: Query handler response message: Error during operation execution: Dataset temperature_data already exists.
2020-02-15 20:27:02 [Client]:ERROR: Query handler response message: Error during operation execution: temperatures_tar TAR already exists.
2020-02-15 20:27:02 [Client]:ERROR: Query handler response message: Error during operation execution: This new subtar definition intersects with already existing subtar!


Abaixo verificamos se os dados foram corretamente registrados no SAVIME.

In [58]:
with pysavime.Client(host=savime_host, port=savime_port, raise_silent_error=True) as client:
    response = client.execute(pysavime.operator.select(tar))[0]
    
is_the_same = np.isclose(response.attrs['temperature'].reshape(squeezed_array.shape),squeezed_array).all()
print('Checagem:', is_the_same)

Checagem: True


O próximo passo é executar o comando PREDICT.

In [59]:
dims_spec = [(group_dim.name, num_groups),
             (time_step_dim.name, num_time_steps - 1),
             (pos_x_dim.name, num_pos_x),
             (pos_y_dim.name, num_pos_y)]

# Vamos selecionar apenas os 9 primeiros instantes de tempo
cmd = pysavime.operator.subset(tar, time_step_dim.name, 0, 8)

with pysavime.Client(host=savime_host, port=savime_port, raise_silent_error=True) as client:
    client.execute(pysavime.operator.register_model(model_name=chosen_model_name, model_tar=tar.name,
                                            input_attribute=temperature.name, dim_specification=dims_spec))
    
    response = client.execute(pysavime.operator.predict(tar=cmd, model_name=chosen_model_name,
                                                input_attribute=temperature.name))[0]

In [60]:
pandas_converter = DataVariableBlockConverter('pandas')
pandas_converter(response)

op_result  temperature
group pos_x pos_y time_step                        
0     0     0     0          23.864020    24.249994
            1     0          24.364020    24.749994
            2     0          24.564032    24.950006
            3     0          24.664038    25.050012
            4     0          24.864020    25.249994
...                                ...          ...
399   34    35    8          21.432570    21.450006
            36    8          21.463592    21.649988
            37    8          21.422684    21.749994
            38    8          21.428188    21.850000
            39    8          21.484580    21.950006

[5040000 rows x 2 columns]